# Homework

#### Explain the bias-variance tradeoff.

# your code here - extra credit ISLR pages 33-37
stop('Not implemented.')

#### Discuss the pros and cons of using the BIC to select a model.

Pro - It is computationally simple to use, it helps select a model, aids in choosing the best fitting model.
Cons - The approximation is only valid for sample size n much larger than the number k of parameters in the model.
The BIC cannot handle complex collections of models as in the variable selection problem in high-dimension.

# Model Selection on a Classification Model

In [11]:
seeds.data = read.csv("seeds_dataset.csv")

In [15]:
seeds.glm = glm("Target ~ 1 + Area + Perimeter + Compactness + Length + Width + Asymmetry + Grove", data = seeds.data)
summary(seeds.glm)


Call:
glm(formula = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width + Asymmetry + Grove", 
    data = seeds.data)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.31887  -0.24382  -0.01955   0.24295   1.21277  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  55.64792    7.54634   7.374 4.90e-12 ***
Area          1.56488    0.26256   5.960 1.20e-08 ***
Perimeter    -3.41870    0.54156  -6.313 1.87e-09 ***
Compactness -31.97971    5.33149  -5.998 9.81e-09 ***
Length       -2.14741    0.46110  -4.657 5.99e-06 ***
Width         0.24688    0.78939   0.313    0.755    
Asymmetry     0.10838    0.02327   4.658 5.97e-06 ***
Grove         2.15089    0.20559  10.462  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.1660733)

    Null deviance: 130.99  on 198  degrees of freedom
Residual deviance:  31.72  on 191  degrees of freedom
  (1

## The Log-Likelihood

Without going too far into the math, we can think of the log-likelihood as a **likelihood function** telling us how likely a model is given the data. 

This value is not human interpretable but is useful as a comparison.

In [16]:
logLik(seeds.glm)

'log Lik.' -99.6512 (df=9)

"All models are wrong, but some are useful." - George Box

We might be concerned with one additional property - the **complexity** of the model. 

##### William of Occam

[**Occam's razor**](https://en.wikipedia.org/wiki/Occam's_razor) is the problem-solving principle that, when presented with competing hypothetical answers to a problem, one should select the one that makes the fewest assumptions.

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ab/William_of_Ockham_-_Logica_1341.jpg" width=400px>

We can represent this idea of complexity in terms of both the number of features we use and the amount of data.

## Bayesian Information Criterion

https://en.wikipedia.org/wiki/Bayesian_information_criterion

The BIC is formally defined as

$$ \mathrm{BIC} = {\log(n)k - 2\log({\widehat L})}. $$

where

- $\widehat L$ = the maximized value of the likelihood function of the model $M$
- $x$ = the observed data
- $n$ = the number of data points in $x$, the number of observations, or equivalently, the sample size;
- $k$ = the number of parameters estimated by the model. For example, in multiple linear regression, the estimated parameters are the intercept, the $q$ slope parameters, and the constant variance of the errors; thus, $k = q + 2$.


It might help us to think of it as 

$$ \mathrm{BIC} = \text{complexity}-\text{likelihood}$$

In [17]:
BIC(seeds.glm)

[1] 246.9422

In [19]:
n = length(seeds.glm$fitted.values)
p = length(coefficients(seeds.glm))

likelihood = 2 * logLik(seeds.glm)
complexity = log(n)*(p+1)

bic = complexity - likelihood
bic

'log Lik.' 246.9422 (df=9)

In [21]:
BIC_of_model = function (model) {
    n = length(model$fitted.values)
    p = length(coefficients(model))

    likelihood = 2 * logLik(model)
    complexity = log(n)*(p+1)

    bic = complexity - likelihood
    return(bic)
}

In [22]:
BIC_of_model(seeds.glm)

'log Lik.' 246.9422 (df=9)

## Model Selection

Here, we choose the optimal model by removing features one by one.

In [23]:
model_1  = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width + Asymmetry + Grove"
model_2a = "Target ~ 1 + Area +             Compactness + Length + Width + Asymmetry + Grove"
model_2b = "Target ~ 1 + Area + Perimeter +               Length + Width + Asymmetry + Grove"
model_2c = "Target ~ 1 + Area + Perimeter + Compactness +          Width + Asymmetry + Grove"
model_2d = "Target ~ 1 + Area + Perimeter + Compactness + Length +         Asymmetry + Grove"
model_2e = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width +             Grove"
model_2f = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width + Asymmetry        "

In [24]:
seeds.glm.1 = glm(model_1, data=seeds.data)
seeds.glm.2a = glm(model_2a, data=seeds.data)
seeds.glm.2b = glm(model_2b, data=seeds.data)
seeds.glm.2c = glm(model_2c, data=seeds.data)
seeds.glm.2d = glm(model_2d, data=seeds.data)
seeds.glm.2e = glm(model_2e, data=seeds.data)
seeds.glm.2f = glm(model_2f, data=seeds.data)

In [25]:
print(c('model_1', BIC_of_model(seeds.glm.1)))
print(c('model_2a', BIC_of_model(seeds.glm.2a )))
print(c('model_2b', BIC_of_model(seeds.glm.2b )))
print(c('model_2c', BIC_of_model(seeds.glm.2c )))
print(c('model_2d', BIC_of_model(seeds.glm.2d )))
print(c('model_2e', BIC_of_model(seeds.glm.2e )))
print(c('model_2f', BIC_of_model(seeds.glm.2f )))

[1] "model_1"          "246.942152387597"
[1] "model_2a"         "279.358145369205"
[1] "model_2b"         "334.008094594033"
[1] "model_2c"         "263.052551365895"
[1] "model_2d"         "241.750726273439"
[1] "model_2e"        "263.05926376933"
[1] "model_2f"         "458.937135824306"


In [26]:
print(c('model_1', BIC(seeds.glm.1)))
print(c('model_2a', BIC(seeds.glm.2a )))
print(c('model_2b', BIC(seeds.glm.2b )))
print(c('model_2c', BIC(seeds.glm.2c )))
print(c('model_2d', BIC(seeds.glm.2d )))
print(c('model_2e', BIC(seeds.glm.2e )))
print(c('model_2f', BIC(seeds.glm.2f )))

[1] "model_1"          "246.942152387597"
[1] "model_2a"         "279.358145369205"
[1] "model_2b"         "334.008094594033"
[1] "model_2c"         "263.052551365895"
[1] "model_2d"         "241.750726273439"
[1] "model_2e"        "263.05926376933"
[1] "model_2f"         "458.937135824306"


In [30]:
model_1  = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width + Asymmetry + Grove"
model_2a = "Target ~ 1 + Area +             Compactness + Length + Width + Asymmetry + Grove"
model_2c = "Target ~ 1 + Area + Perimeter + Compactness +          Width + Asymmetry + Grove"
model_3a = "Target ~ 1 + Area + Perimeter +                      + Width + Asymmetry + Grove"
model_3b = "Target ~ 1 + Area + Perimeter + Compactness +                  Asymmetry + Grove"
model_3c = "Target ~ 1 + Area + Perimeter + Compactness + Length +         Asymmetry        "
model_3d = "Target ~ 1 + Area + Perimeter + Compactness + Length + Width                    "

In [31]:
seeds.glm.3a = glm(model_3a, data=seeds.data)
seeds.glm.3b = glm(model_3b, data=seeds.data)
seeds.glm.3c = glm(model_3c, data=seeds.data)
seeds.glm.3d = glm(model_3d, data=seeds.data)

In [33]:
print(c('model_1', BIC(seeds.glm.1)))
print(c('model_2a', BIC(seeds.glm.2c )))
print(c('model_2c', BIC(seeds.glm.2c )))
print(c('model_3a', BIC(seeds.glm.3a )))
print(c('model_3b', BIC(seeds.glm.3b )))
print(c('model_3c', BIC(seeds.glm.3c )))
print(c('model_3d', BIC(seeds.glm.3c )))

[1] "model_1"          "246.942152387597"
[1] "model_2a"         "263.052551365895"
[1] "model_2c"         "263.052551365895"
[1] "model_3a"         "376.393818585246"
[1] "model_3b"         "259.575072490043"
[1] "model_3c"         "454.680752932735"
[1] "model_3d"         "454.680752932735"
